## Microsegmentation_prep:

The purpose of this step is to prepare the data to perform a microsegmentation.

In [56]:
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

In [2]:
driver = 'mysql+pymysql:'
user = 'lordchava'
password = '19693903'
ip = '104.155.143.225'
database = 'final_project'

In [3]:
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

### Importing transactions table

In [10]:
transactions_df = pd.read_csv("../../data_final_project/transactions_clean.csv")

In [11]:
transactions_df.head()

,Unnamed: 0,HOUSEHOLD_KEY,BASKET_ID,WEEK_NO,DAY,TRANS_TIME,STORE_ID,PRODUCT_ID,QUANTITY,SALES_VALUE,RETAIL_DISC,COUPON_DISC,COUPON_MATCH_DISC,SHELF_PRICE,CLIENT_PRICE,%_TOTAL_DISCOUNT,%_LOYALTY_DISCOUNT,%_COUPON_DISCOUNT
0,0,2375,26984851472,1,1,1631,364,1004906,1,1.39,-0.60,0.0,0.0,1.99,1.39,0.301508,0.301508,0.0
1,1,2375,26984851472,1,1,1631,364,1033142,1,0.82,0.00,0.0,0.0,0.82,0.82,0.000000,0.000000,0.0
2,2,2375,26984851472,1,1,1631,364,1036325,1,0.99,-0.30,0.0,0.0,1.29,0.99,0.232558,0.232558,0.0
3,3,2375,26984851472,1,1,1631,364,1082185,1,1.21,0.00,0.0,0.0,1.21,1.21,0.000000,0.000000,0.0
4,4,2375,26984851472,1,1,1631,364,8160430,1,1.50,-0.39,0.0,0.0,1.89,1.50,0.206349,0.206349,0.0


In [13]:
transactions_df = transactions_df.drop("Unnamed: 0", axis=1)

### Reordering columns

In [14]:
transactions_df.columns

Index(['HOUSEHOLD_KEY', 'BASKET_ID', 'WEEK_NO', 'DAY', 'TRANS_TIME',
       'STORE_ID', 'PRODUCT_ID', 'QUANTITY', 'SALES_VALUE', 'RETAIL_DISC',
       'COUPON_DISC', 'COUPON_MATCH_DISC', 'SHELF_PRICE', 'CLIENT_PRICE',
       '%_TOTAL_DISCOUNT', '%_LOYALTY_DISCOUNT', '%_COUPON_DISCOUNT'],
      dtype='object')

In [15]:
transactions_columns = ['HOUSEHOLD_KEY', 'BASKET_ID', 'WEEK_NO', 'DAY', 'TRANS_TIME',
       'STORE_ID', 'PRODUCT_ID', 'QUANTITY', 'SHELF_PRICE', 'SALES_VALUE', 'RETAIL_DISC',
       'COUPON_DISC', 'COUPON_MATCH_DISC', 'CLIENT_PRICE',
       '%_TOTAL_DISCOUNT', '%_LOYALTY_DISCOUNT', '%_COUPON_DISCOUNT']

In [16]:
transactions_df = transactions_df[transactions_columns]

### Checking column types

In [17]:
transactions_df.dtypes

HOUSEHOLD_KEY           int64
BASKET_ID               int64
WEEK_NO                 int64
DAY                     int64
TRANS_TIME              int64
STORE_ID                int64
PRODUCT_ID              int64
QUANTITY                int64
SHELF_PRICE           float64
SALES_VALUE           float64
RETAIL_DISC           float64
COUPON_DISC           float64
COUPON_MATCH_DISC     float64
CLIENT_PRICE          float64
%_TOTAL_DISCOUNT      float64
%_LOYALTY_DISCOUNT    float64
%_COUPON_DISCOUNT     float64
dtype: object

### Creating a synthetic basket identifier called "PURCHASE_ID" to complement the existing identifier

In [18]:
transactions_df["PURCHASE_ID"] = transactions_df[["DAY", "TRANS_TIME", "HOUSEHOLD_KEY"]].apply(lambda x: int("".join(map(str, x))), axis=1)

In [21]:
transactions_df.head()

,HOUSEHOLD_KEY,BASKET_ID,WEEK_NO,DAY,TRANS_TIME,STORE_ID,PRODUCT_ID,QUANTITY,SHELF_PRICE,SALES_VALUE,RETAIL_DISC,COUPON_DISC,COUPON_MATCH_DISC,CLIENT_PRICE,%_TOTAL_DISCOUNT,%_LOYALTY_DISCOUNT,%_COUPON_DISCOUNT,PURCHASE_ID
0,2375,26984851472,1,1,1631,364,1004906,1,1.99,1.39,-0.60,0.0,0.0,1.39,0.301508,0.301508,0.0,116312375
1,2375,26984851472,1,1,1631,364,1033142,1,0.82,0.82,0.00,0.0,0.0,0.82,0.000000,0.000000,0.0,116312375
2,2375,26984851472,1,1,1631,364,1036325,1,1.29,0.99,-0.30,0.0,0.0,0.99,0.232558,0.232558,0.0,116312375
3,2375,26984851472,1,1,1631,364,1082185,1,1.21,1.21,0.00,0.0,0.0,1.21,0.000000,0.000000,0.0,116312375
4,2375,26984851472,1,1,1631,364,8160430,1,1.89,1.50,-0.39,0.0,0.0,1.50,0.206349,0.206349,0.0,116312375


### [comment better] 2 adjustments to our original dataframes

comment. price adjustment

In [28]:
transactions_red_df = transactions_df[transactions_df["SHELF_PRICE"]>=1]

cpomment. demographisc adjustment

In [39]:
demo_df = pd.read_csv("../../data_final_project/hh_demographic_data.csv")

In [40]:
demo_df.head()

,HOUSEHOLD_KEY,AGE_DESC,INCOME_DESC,ADULTS_NUM,KIDS_NUM,NUMEROUS_FAM,SINGLE_GENDER
0,1,65+,35-49K,2,0,0,NaN
1,7,45-54,50-74K,2,0,0,NaN
2,8,25-34,25-34K,2,1,0,NaN
3,13,25-34,75-99K,2,2,0,NaN
4,16,45-54,50-74K,1,0,0,Single Female


In [41]:
transactions_red_df.head()

,HOUSEHOLD_KEY,BASKET_ID,WEEK_NO,DAY,TRANS_TIME,STORE_ID,PRODUCT_ID,QUANTITY,SHELF_PRICE,SALES_VALUE,RETAIL_DISC,COUPON_DISC,COUPON_MATCH_DISC,CLIENT_PRICE,%_TOTAL_DISCOUNT,%_LOYALTY_DISCOUNT,%_COUPON_DISCOUNT,PURCHASE_ID
0,2375,26984851472,1,1,1631,364,1004906,1,1.99,1.39,-0.60,0.0,0.0,1.39,0.301508,0.301508,0.0,116312375
2,2375,26984851472,1,1,1631,364,1036325,1,1.29,0.99,-0.30,0.0,0.0,0.99,0.232558,0.232558,0.0,116312375
3,2375,26984851472,1,1,1631,364,1082185,1,1.21,1.21,0.00,0.0,0.0,1.21,0.000000,0.000000,0.0,116312375
4,2375,26984851472,1,1,1631,364,8160430,1,1.89,1.50,-0.39,0.0,0.0,1.50,0.206349,0.206349,0.0,116312375
5,2375,26984851516,1,1,1642,364,826249,2,2.58,1.98,-0.60,0.0,0.0,1.98,0.232558,0.232558,0.0,116422375


Joining transactions with demographics in order to just have the transactions of the customers for whom we have demographics info about

In [42]:
transactions_red_df = demo_df.merge(transactions_red_df, how="left", on="HOUSEHOLD_KEY")

In [47]:
transactions_red_df = transactions_red_df.drop(["AGE_DESC", "INCOME_DESC", "ADULTS_NUM", "KIDS_NUM",
                                               "NUMEROUS_FAM", "SINGLE_GENDER"], axis=1)

In [48]:
transactions_red_df.head()

,HOUSEHOLD_KEY,BASKET_ID,WEEK_NO,DAY,TRANS_TIME,STORE_ID,PRODUCT_ID,QUANTITY,SHELF_PRICE,SALES_VALUE,RETAIL_DISC,COUPON_DISC,COUPON_MATCH_DISC,CLIENT_PRICE,%_TOTAL_DISCOUNT,%_LOYALTY_DISCOUNT,%_COUPON_DISCOUNT,PURCHASE_ID
0,1,27601281299,8,51,1456,436,825123,1,3.99,3.99,0.00,0.0,0.0,3.99,0.000000,0.000000,0.0,5114561
1,1,27601281299,8,51,1456,436,831447,1,2.99,2.99,0.00,0.0,0.0,2.99,0.000000,0.000000,0.0,5114561
2,1,27601281299,8,51,1456,436,840361,1,1.39,1.09,-0.30,0.0,0.0,1.09,0.215827,0.215827,0.0,5114561
3,1,27601281299,8,51,1456,436,845307,1,4.33,3.71,-0.62,0.0,0.0,3.71,0.143187,0.143187,0.0,5114561
4,1,27601281299,8,51,1456,436,852014,1,3.99,2.79,-1.20,0.0,0.0,2.79,0.300752,0.300752,0.0,5114561


### Products

In [69]:
products_df = pd.read_csv("../../data_final_project/products_data.csv")

In [70]:
products_df.head()

,PRODUCT_ID,MANUFACTURER,BRAND,DEPARTMENT,SUB_DEPARTMENT,COMMODITY_DESC,SUB_COMMODITY_DESC
0,25671,2,National,GROCERY,FROZEN_GROCERY,FRZN ICE,ICE - CRUSHED/CUBED
1,26081,2,National,MISC. TRANS.,MISCELLANEOUS,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION
2,26093,69,Private,GROCERY,BAKED,BREAD,BREAD:ITALIAN/FRENCH
3,26190,69,Private,GROCERY,FRUITS_VEGETABLES,FRUIT - SHELF STABLE,APPLE SAUCE
4,26355,69,Private,GROCERY,SNACKS,COOKIES/CONES,SPECIALTY COOKIES


In [71]:
transactions_products = transactions_red_df.merge(products_df, how="left", on="PRODUCT_ID")

In [72]:
transactions_products.head()

,HOUSEHOLD_KEY,BASKET_ID,WEEK_NO,DAY,TRANS_TIME,STORE_ID,PRODUCT_ID,QUANTITY,SHELF_PRICE,SALES_VALUE,RETAIL_DISC,COUPON_DISC,COUPON_MATCH_DISC,CLIENT_PRICE,%_TOTAL_DISCOUNT,%_LOYALTY_DISCOUNT,%_COUPON_DISCOUNT,PURCHASE_ID,MANUFACTURER,BRAND,DEPARTMENT,SUB_DEPARTMENT,COMMODITY_DESC,SUB_COMMODITY_DESC
0,1,27601281299,8,51,1456,436,825123,1,3.99,3.99,0.00,0.0,0.0,3.99,0.000000,0.000000,0.0,5114561,1179,National,GROCERY,CONDIMENTS,SALD DRSNG/SNDWCH SPRD,SEMI-SOLID SALAD DRESSING MAY
1,1,27601281299,8,51,1456,436,831447,1,2.99,2.99,0.00,0.0,0.0,2.99,0.000000,0.000000,0.0,5114561,317,National,GROCERY,DAIRY,CHEESE,SHREDDED CHEESE
2,1,27601281299,8,51,1456,436,840361,1,1.39,1.09,-0.30,0.0,0.0,1.09,0.215827,0.215827,0.0,5114561,69,Private,GROCERY,BREAKFAST,EGGS,EGGS - LARGE
3,1,27601281299,8,51,1456,436,845307,1,4.33,3.71,-0.62,0.0,0.0,3.71,0.143187,0.143187,0.0,5114561,4084,National,DELI,DELI,DELI MEATS,MEAT: TURKEY BULK
4,1,27601281299,8,51,1456,436,852014,1,3.99,2.79,-1.20,0.0,0.0,2.79,0.300752,0.300752,0.0,5114561,69,Private,GROCERY,DAIRY,ICE CREAM/MILK/SHERBTS,TRADITIONAL


In [79]:
transactions_products_pv = transactions_products.pivot_table(index=["HOUSEHOLD_KEY"], columns=["SUB_DEPARTMENT"],
                                  values=["SHELF_PRICE"], fill_value=0, aggfunc="mean")

In [80]:
transactions_products_pv.head()

SHELF_PRICE                                            \
SUB_DEPARTMENT     APPAREL AUTOMOTIVE  BABY_CARE     BAKED BREAKFAST   
HOUSEHOLD_KEY                                                          
1                 1.790000        0.0   0.000000  2.835926  2.775641   
7                 0.000000        0.0   3.429877  2.400339  2.616375   
8                 8.353333        0.0   6.365957  2.377273  2.629888   
13                3.190000        0.0  17.218675  2.848733  2.738939   
16                0.000000        0.0   0.000000  2.475417  3.400952   

                                                                       \
SUB_DEPARTMENT CLEANING_PROD CONDIMENTS COSMETICS     DAIRY      DELI   
HOUSEHOLD_KEY                                                           
1                   3.960000   2.915476  0.000000  2.787355  3.318382   
7                   4.485556   2.930435  7.203125  2.846696  5.128125   
8                   5.302245   2.770924  5.710000  2.908579  5.391000   
13                  4.891833   2.555795  8.081429  5.048052  3.348387   
16                  4.265000   2.658636  0.000000  2.223333  3.695000   

                                                                        \
SUB_DEPARTMENT DRINKS_JUICES DRUGSTORE       ECO  ELECTRIC ELECTRONICS   
HOUSEHOLD_KEY                                                            
1                   5.169588  6.740000  2.611739  4.693333       0.000   
7                   3.075325  5.773333  2.990000  3.540000       3.000   
8                   3.900484  6.297857  5.990000  3.130000      12.290   
13                  3.950048  6.418222  0.000000  6.917273       9.166   
16                  3.148889  4.535000  0.000000  2.740000       0.000   

                                                                          \
SUB_DEPARTMENT     FLORAL FROZEN_GROCERY FRUITS_VEGETABLES GARDEN CENTER   
HOUSEHOLD_KEY                                                              
1                7.990000       3.034086          2.192415          0.00   
7                4.990000       4.089278          2.588736          0.00   
8               12.083636       3.063184          2.837032         13.34   
13              11.737500       4.298830          3.149528          0.00   
16               0.000000       3.514286          2.511707          0.00   

                                                                       \
SUB_DEPARTMENT GARDEN_CENTER GRAINS_PASTA_SIDES HISPANIC_FOOD    HOME   
HOUSEHOLD_KEY                                                           
1                       0.00           1.990748      1.090000   1.190   
7                       4.29           2.441200      5.670000  12.485   
8                       3.49           2.678359      3.314167   4.520   
13                      3.50           3.570852      0.000000   0.000   
16                     14.99           2.770000      1.390000   0.000   

                                                                    \
SUB_DEPARTMENT HOUSEWARES      KIDS     KIOSK    LIQUOUR      MEAT   
HOUSEHOLD_KEY                                                        
1                4.249747  2.816737  0.000000   0.000000  3.540000   
7                2.828571  2.911600  3.523684   4.990000  4.745309   
8                4.313906  3.100000  3.423333   7.634828  5.676181   
13               5.151546  3.981475  4.439062  11.110000  5.304149   
16               3.780000  1.683500  0.000000   5.990000  5.370909   

                                                                            \
SUB_DEPARTMENT MISCELLANEOUS OTHERS PERSONAL_CARE PETS_SUPPLIES RESTAURANT   
HOUSEHOLD_KEY                                                                
1                  10.000000    0.0      3.962083      0.000000       1.39   
7                   1.750000    0.0      4.305000      2.919000       0.00   
8                  21.730000   22.5      3.764783      7.823333       0.00   
13                 45.983824    0.0  

In [96]:
transactions_products_df = pd.DataFrame(transactions_products_pv.to_records())

In [98]:
transactions_products_df.columns = [hdr.replace("('SHELF_PRICE', ", "").replace(")", "").replace("'", "") \
                     for hdr in transactions_products_df]

In [99]:
transactions_products_df.head()

,HOUSEHOLD_KEY,APPAREL,AUTOMOTIVE,BABY_CARE,BAKED,BREAKFAST,CLEANING_PROD,CONDIMENTS,COSMETICS,DAIRY,DELI,DRINKS_JUICES,DRUGSTORE,ECO,ELECTRIC,ELECTRONICS,FLORAL,FROZEN_GROCERY,FRUITS_VEGETABLES,GARDEN CENTER,GARDEN_CENTER,GRAINS_PASTA_SIDES,HISPANIC_FOOD,HOME,HOUSEWARES,KIDS,KIOSK,LIQUOUR,MEAT,MISCELLANEOUS,OTHERS,PERSONAL_CARE,PETS_SUPPLIES,RESTAURANT,SEAFOOD,SEASONS,SMOKING,SNACKS,SPORTS_MISC
0,1,1.790000,0.0,0.000000,2.835926,2.775641,3.960000,2.915476,0.000000,2.787355,3.318382,5.169588,6.740000,2.611739,4.693333,0.000,7.990000,3.034086,2.192415,0.00,0.00,1.990748,1.090000,1.190,4.249747,2.816737,0.000000,0.000000,3.540000,10.000000,0.0,3.962083,0.000000,1.39,3.940000,4.025,5.88,2.878698,6.170000
1,7,0.000000,0.0,3.429877,2.400339,2.616375,4.485556,2.930435,7.203125,2.846696,5.128125,3.075325,5.773333,2.990000,3.540000,3.000,4.990000,4.089278,2.588736,0.00,4.29,2.441200,5.670000,12.485,2.828571,2.911600,3.523684,4.990000,4.745309,1.750000,0.0,4.305000,2.919000,0.00,4.060000,3.115,0.00,2.556600,0.000000
2,8,8.353333,0.0,6.365957,2.377273,2.629888,5.302245,2.770924,5.710000,2.908579,5.391000,3.900484,6.297857,5.990000,3.130000,12.290,12.083636,3.063184,2.837032,13.34,3.49,2.678359,3.314167,4.520,4.313906,3.100000,3.423333,7.634828,5.676181,21.730000,22.5,3.764783,7.823333,0.00,4.900000,6.990,0.00,2.944667,9.990000
3,13,3.190000,0.0,17.218675,2.848733,2.738939,4.891833,2.555795,8.081429,5.048052,3.348387,3.950048,6.418222,0.000000,6.917273,9.166,11.737500,4.298830,3.149528,0.00,3.50,3.570852,0.000000,0.000,5.151546,3.981475,4.439062,11.110000,5.304149,45.983824,0.0,5.705902,7.987045,0.00,6.178571,7.505,0.00,3.097159,9.740769
4,16,0.000000,0.0,0.000000,2.475417,3.400952,4.265000,2.658636,0.000000,2.223333,3.695000,3.148889,4.535000,0.000000,2.740000,0.000,0.000000,3.514286,2.511707,0.00,14.99,2.770000,1.390000,0.000,3.780000,1.683500,0.000000,5.990000,5.370909,24.921250,0.0,4.090000,5.530000,0.00,4.380000,1.790,0.00,3.103636,0.000000


In [113]:
transactions_products_df.to_csv('../../data_final_project/transactions_products_df.csv', index=False)

### Further join with clusters

In [101]:
macro_cluster = pd.read_csv("../../data_final_project/macro_cluster_df.csv")

In [107]:
pre_micro = transactions_products_df.merge(macro_cluster, how="right", on="HOUSEHOLD_KEY")

### Slicing the data to obtain only the most relevant sub_departments for the analysis.

In [109]:
pre_micro.columns

Index(['HOUSEHOLD_KEY', 'APPAREL', 'AUTOMOTIVE', 'BABY_CARE', 'BAKED',
       'BREAKFAST', 'CLEANING_PROD', 'CONDIMENTS', 'COSMETICS', 'DAIRY',
       'DELI', 'DRINKS_JUICES', 'DRUGSTORE', 'ECO', 'ELECTRIC', 'ELECTRONICS',
       'FLORAL', 'FROZEN_GROCERY', 'FRUITS_VEGETABLES', 'GARDEN CENTER',
       'GARDEN_CENTER', 'GRAINS_PASTA_SIDES', 'HISPANIC_FOOD', 'HOME',
       'HOUSEWARES', 'KIDS', 'KIOSK', 'LIQUOUR', 'MEAT', 'MISCELLANEOUS',
       'OTHERS', 'PERSONAL_CARE', 'PETS_SUPPLIES', 'RESTAURANT', 'SEAFOOD',
       'SEASONS', 'SMOKING', 'SNACKS', 'SPORTS_MISC', 'baskets',
       'avg_unit_price', 'avg_qty_per_basket', 'total_expenditure', 'cluster',
       'AGE_DESC', 'INCOME_DESC', 'ADULTS_NUM', 'KIDS_NUM', 'NUMEROUS_FAM',
       'SINGLE_GENDER', 'AGE_MED', 'INCOME_LEVEL', 'GENDER_CAT'],
      dtype='object')

In [111]:
pre_micro = pre_micro.drop(["baskets", 'avg_unit_price', 'avg_qty_per_basket', 'total_expenditure',
       'AGE_DESC', 'INCOME_DESC', 'ADULTS_NUM', 'KIDS_NUM', 'NUMEROUS_FAM',
       'SINGLE_GENDER', 'AGE_MED', 'INCOME_LEVEL', 'GENDER_CAT'], axis=1)

In [112]:
pre_micro.head()

,HOUSEHOLD_KEY,APPAREL,AUTOMOTIVE,BABY_CARE,BAKED,BREAKFAST,CLEANING_PROD,CONDIMENTS,COSMETICS,DAIRY,DELI,DRINKS_JUICES,DRUGSTORE,ECO,ELECTRIC,ELECTRONICS,FLORAL,FROZEN_GROCERY,FRUITS_VEGETABLES,GARDEN CENTER,GARDEN_CENTER,GRAINS_PASTA_SIDES,HISPANIC_FOOD,HOME,HOUSEWARES,KIDS,KIOSK,LIQUOUR,MEAT,MISCELLANEOUS,OTHERS,PERSONAL_CARE,PETS_SUPPLIES,RESTAURANT,SEAFOOD,SEASONS,SMOKING,SNACKS,SPORTS_MISC,cluster
0,1,1.79,0.0,0.000000,2.835926,2.775641,3.960000,2.915476,0.000000,2.787355,3.318382,5.169588,6.740000,2.611739,4.693333,0.000,7.99,3.034086,2.192415,0.0,0.00,1.990748,1.090000,1.190,4.249747,2.816737,0.000000,0.000000,3.540000,10.000000,0.0,3.962083,0.0000,1.390000,3.940000,4.025,5.880000,2.878698,6.17,3
1,7,0.00,0.0,3.429877,2.400339,2.616375,4.485556,2.930435,7.203125,2.846696,5.128125,3.075325,5.773333,2.990000,3.540000,3.000,4.99,4.089278,2.588736,0.0,4.29,2.441200,5.670000,12.485,2.828571,2.911600,3.523684,4.990000,4.745309,1.750000,0.0,4.305000,2.9190,0.000000,4.060000,3.115,0.000000,2.556600,0.00,2
2,20,0.00,0.0,7.990000,3.568809,3.048807,3.740000,2.347326,1.990000,2.742246,0.000000,5.351111,5.014800,0.000000,3.990000,0.000,0.00,3.354453,2.474306,0.0,0.00,2.698041,1.985556,0.000,2.096316,2.119444,2.674615,7.906667,3.818632,25.222708,23.8,2.655000,7.7275,7.666667,3.764706,5.990,5.157843,3.006389,1.89,2
3,25,2.00,0.0,2.190000,1.804696,2.864318,2.832857,1.933611,0.000000,4.021583,5.121250,4.221304,5.830000,1.000000,1.340000,10.665,0.00,2.430588,2.456418,0.0,0.00,1.992647,0.000000,2.250,3.195263,2.716667,2.018000,10.374615,3.560373,13.462857,0.0,4.090000,0.0000,0.000000,4.316000,1.000,41.105263,2.585714,1.49,2
4,27,2.84,0.0,0.000000,2.205714,2.108571,3.480000,1.463673,1.045000,1.797483,1.813400,1.672558,3.475455,1.490000,0.000000,0.000,0.00,2.604375,2.077692,0.0,0.00,2.116875,0.000000,0.000,1.767857,1.491778,2.458421,4.196424,3.298495,5.361538,0.0,3.203333,2.4500,0.000000,3.560000,1.090,4.226196,2.139000,0.00,4


In [114]:
pre_micro.to_csv('../../data_final_project/pre_micro_df.csv', index=False)